In [5]:
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException
import pandas as pd
from pathlib import Path
import unicodedata
import re
class DriverInit:
    ACCEPT_COOKIES_XPATH = "//button[@id='L2AGLb']"
    REVIEWS_TAB_XPATH = "//span[text()='Reviews']"
    GOOGLE_SEARCH_XPATH = "//textarea[@class='gLFyf']"


def __init__(self):
    self.driver = webdriver.Chrome()
    self.wait = WebDriverWait(self.driver, 10)
    self.short_wait = WebDriverWait(self.driver, 5, poll_frequency=0.5)
    self.driver.maximize_window()


def navigate_to_url(self, url):
    self.driver.get(url)
    accept_cookies = self.wait.until(EC.element_to_be_clickable((By.XPATH, self.ACCEPT_COOKIES_XPATH)))
    accept_cookies.click()


def navigate_to_reviews_tab(self):
    reviews_tab = self.wait.until(EC.element_to_be_clickable((By.XPATH, self.REVIEWS_TAB_XPATH)))
    reviews_tab.click()


def close_browser(self):
    self.driver.quit()


def go_to_restaurant_list(self):
    self.wait.until(EC.element_to_be_clickable((By.XPATH, self.GOOGLE_SEARCH_XPATH))).send_keys('restorani')

In [6]:
city='Zagreb'

In [3]:
url = 'https://www.google.hr'
ACCEPT_COOKIES_XPATH = "//button[@id='L2AGLb']"
REVIEWS_TAB_XPATH = "//span[text()='Reviews']"
GOOGLE_SEARCH_XPATH = "//textarea[@class='gLFyf']"
GO_TO_RESTAURANT_LIST_XPATH = "//span[text()='Više mjesta']"

In [4]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
short_wait = WebDriverWait(driver, 5, poll_frequency=0.5)
driver.maximize_window()
driver.get(url)

In [5]:
accept_cookies = wait.until(EC.element_to_be_clickable((By.XPATH, ACCEPT_COOKIES_XPATH)))
accept_cookies.click()

In [6]:
google_search_bar = wait.until(EC.element_to_be_clickable((By.XPATH, GOOGLE_SEARCH_XPATH)))
google_search_bar.send_keys(f'restorani {city}')
google_search_bar.submit()

In [7]:
restaurant_list = wait.until(EC.element_to_be_clickable((By.XPATH, GO_TO_RESTAURANT_LIST_XPATH)))
restaurant_list.click()

In [8]:
MORE_PLACES_XPATH = "//span[text()='Više rezultata']"
LOADING_ICON_XPATH = "//div[@data-hveid='CAQQAw']"
TITLE_XPATH = "//span[@class='OSrXXb e62wic']"

In [9]:
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def get_last_element_text(driver, xpath):
    """
    Retrieves the text of the last element matching the given XPath.
    Returns None if no elements are found.
    """
    try:
        elements = driver.find_elements(By.XPATH, xpath)
        if elements:
            return elements[-1].text
        else:
            return None
    except NoSuchElementException:
        return None

last_unique_element_text = None

def wait_for_loading_complete(driver, loading_icon_xpath, timeout=10):
    """
    Waits for the loading icon to disappear and reappear if necessary,
    indicating that loading sequences have completed.
    """
    end_time = time.time() + timeout
    while time.time() < end_time:
        try:
            # Check for the presence of the loading icon.
            loading_icon_present = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.XPATH, loading_icon_xpath))
            )
            if loading_icon_present:
                # Wait for the loading icon to disappear.
                WebDriverWait(driver, 5).until(
                    EC.invisibility_of_element_located((By.XPATH, loading_icon_xpath))
                )
        except TimeoutException:
            # If the loading icon isn't found or disappears and doesn't reappear, break the loop.
            break




In [46]:
while True:
    try:
        # Wait for the "more places" button to become visible
        more_places_button = wait.until(EC.visibility_of_element_located((By.XPATH, MORE_PLACES_XPATH)))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", more_places_button)

        # Click the button using JavaScript
        driver.execute_script("arguments[0].click();", more_places_button)

        # Dynamically wait for loading to complete
        wait_for_loading_complete(driver, LOADING_ICON_XPATH)

        # Retrieve the text of the last element
        current_last_element_text = get_last_element_text(driver, TITLE_XPATH)

        if current_last_element_text == last_unique_element_text:
            print("No new content loaded. End of list reached.")
            break
        else:
            last_unique_element_text = current_last_element_text

        # Wait for the "more places" button to become visible again
        WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, MORE_PLACES_XPATH)))

    except TimeoutException:
        print("All 'Više mjesta' elements have been clicked, or no more are found.")
        break
    except (ElementNotInteractableException, ElementClickInterceptedException, NoSuchElementException):
        print("The 'Više mjesta' button is no longer interactable or clickable. All content loaded.")
        break

All 'Više mjesta' elements have been clicked, or no more are found.


In [12]:
len(titles)

208

In [13]:
import pandas as pd
from pathlib import Path

# Create a DataFrame
df_titles = pd.DataFrame(titles, columns=['Restaurant Title'])

# Define the city variable
city = 'Zagreb'  # Replace with the actual city name as needed

# Specify the output file path relative to the project's root directory
output_file_path = Path(f'../data/{city}/restaurant_list.csv')

# Ensure the directory exists
output_file_path.parent.mkdir(parents=True, exist_ok=True)

# Save to CSV
df_titles.to_csv(output_file_path, index=False)

# Confirm the save
# Print the absolute path of the file for confirmation
print(f'File saved at {output_file_path.resolve()}')


File saved at C:\Users\ivan.zeljeznjak\DataspellProjects\RedditSentimentAnalysis\data\Zagreb\restaurant_list.csv


In [14]:
df=pd.read_csv(output_file_path)

In [15]:
import pandas as pd
import re
import unicodedata

def clean_name(name):
    # Normalize to handle special characters and convert to lowercase
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode().lower()
    generic_words = ['restoran', 'pizzeria']

    # Remove generic words and handle special cases
    for word in generic_words:
        pattern = r'\b' + re.escape(word) + r'\b'
        name = re.sub(pattern, '', name, flags=re.IGNORECASE)

    # Replace & and - with space to handle cases like "Lari & Penati", "Sofra - Borovje"
    name = name.replace('&', ' ').replace('-', ' ')

    # Remove unwanted characters, keeping spaces and alphanumeric characters
    name = re.sub(r"[^a-z0-9\s]", '', name)

    # Replace multiple spaces with a single space
    name = re.sub(r'\s+', ' ', name)

    # Trim leading and trailing spaces
    name = name.strip()

    # If the name is empty after cleaning, return None
    if not name:
        return None, None

    # Replace spaces with a single underscore for the file name
    name_for_filename = name.replace(' ', '_')

    # Fix casing for the cleaned title
    cleaned_name = ' '.join(name.split()).title()

    return cleaned_name, name_for_filename

# Apply the enhanced cleaning function
df[['Cleaned Title', 'File Name']] = df.apply(lambda x: pd.Series(clean_name(x['Restaurant Title'])), axis=1)

# Drop rows where 'Cleaned Title' is None
df.dropna(subset=['Cleaned Title'], inplace=True)


In [16]:
df

,Restaurant Title,Cleaned Title,File Name
0,Plješivička Klet,Pljesivicka Klet,pljesivicka_klet
1,Vinodol,Vinodol,vinodol
2,Pri Zvoncu,Pri Zvoncu,pri_zvoncu
3,Batak Grill - Rudeš,Batak Grill Rudes,batak_grill_rudes
4,Restoran Taurus,Taurus,taurus
...,...,...,...
203,CHi LE MA Zagreb 吃了吗,Chi Le Ma Zagreb,chi_le_ma_zagreb
204,Esplanade Zagreb Hotel,Esplanade Zagreb Hotel,esplanade_zagreb_hotel
205,3Hugs Vegan Brunch Bar,3Hugs Vegan Brunch Bar,3hugs_vegan_brunch_bar
206,VeganKO,Veganko,veganko


In [17]:
df.iloc[1,0]

'Vinodol'

In [18]:
for index, row in df.iterrows():
    restaurant_name = row['Restaurant Title']
    print(restaurant_name)

Plješivička Klet
Vinodol
Pri Zvoncu
Batak Grill - Rudeš
Restoran Taurus
Restoran Nokturno
Restoran Potkova
Restoran Konoba Didov San - Kajzerica
Restoran "Dubravkin put"
Umami
Restoran Kaiser
Restoran HAS
Restoran Baltazar
Restoran Lido
Restoran Zagreb
Zalogajnica Dolma
VODNJANKA Zagreb
Steakhouse El Toro
Fidel Gastro
Restaurant Balon
Restoran Pupa kod Ljubice
Il Secondo
Restoran Zinfandel's
Maredo Zagreb
Restoran Botanist
Restoran "Mali Bar"
Restoran Uspinjača
La Štruk
Restoran "Kod Pere"
Torero Restaurant
Mano2
Restoran Bicko
Restoran Sofra Vrbani
Stari Kotač
Ristorante Carpaccio
R&B Food
Boban caffe & restoran
Il camino grill & pasta bar
Kaptol Zagreb restoran i caffe bar
Konoba Didov San - Gornji Grad
Trilogija Fino&vino
Kaptolska klet
Brenner
Gostionica Ficlek
Bota-Šare Restaurant
American Steak & Grill House Zagreb Restaurant
Takenoko Zagreb
Gostionica Purger
Le Bistro
Pod Zidom
Bistro Točkica
Ponistra
Restoran Bonaca Nova
Restaurant and Caffe "Muzej"
Restoran Bonaca
Stari Fijake

In [69]:
url = 'https://www.google.hr'
ACCEPT_COOKIES_XPATH = "//button[@id='L2AGLb']"
REVIEWS_TAB_XPATH = "//span[text()='Reviews']"
GOOGLE_SEARCH_XPATH = "//textarea[@class='gLFyf']"
RESTAURANT_REVIEWS_REDIRECT_XPATH = '//a[contains(text(), " recenzija")]'
RESTAURANT_REVIEWS_LINK_XPATH = '//a[@class="Ky0SRd"]'

In [70]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 60)
short_wait = WebDriverWait(driver, 60, poll_frequency=0.5)
driver.maximize_window()
driver.get(url)

In [71]:
accept_cookies = wait.until(EC.element_to_be_clickable((By.XPATH, ACCEPT_COOKIES_XPATH)))
accept_cookies.click()

In [72]:
google_search_bar = wait.until(EC.element_to_be_clickable((By.XPATH, GOOGLE_SEARCH_XPATH)))
google_search_bar.send_keys(f'Kavana i restoran Johann Franck {city}')
google_search_bar.submit()

In [73]:
reviews_redirect = wait.until(EC.element_to_be_clickable((By.XPATH , RESTAURANT_REVIEWS_REDIRECT_XPATH)))
reviews_redirect.click()

In [74]:
reviews_link = wait.until(EC.element_to_be_clickable((By.XPATH , RESTAURANT_REVIEWS_LINK_XPATH)))
reviews_link.click()

In [75]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import os

from selenium.webdriver.support.wait import WebDriverWait


class ReviewScraper:
    FIRST_REVIEW_XPATH = ("//*[@id='akp_tsuid_13']/div/div[1]/div/g-sticky-content-container/div/block-component/div"
                          "/div[1]/div/div/div/div[1]/div/div/div[6]/g-flippy-carousel/div/div/ol/li["
                          "3]/span/div/div/div/div[2]/c-wiz/div/div[2]/div[1]/div[1]")
    REVIEW_ELEMENTS_XPATH = "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']"
    TRANSLATE_BUTTON_XPATH = ("//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]/div[3]//button["
                              "contains(@class, 'qGGAec') and .//span[contains(text(), 'Translated by Google')]]")
    MORE_BUTTON_XPATH = "//div[@class='OA1nbd']//a[@role='button' and contains(text(), 'Više')]"
    REVIEW_XPATH = "//div[@class='OA1nbd']"
    RATING_XPATH = ("//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]//div[contains(@aria-label, "
                    "'Rated') and contains(@role, 'img')]")
    #LOADING_INDICATOR = "//div[string-length(@data-loadingmessage) > 0]"
    LOADING_INDICATOR = "//div[@jscontroller='qAKInc'][string-length(@data-loadingmessage) > 0]"
    REVIEW_BLOCK_XPATH = "//div[contains(@class, 'bwb7ce')]"

    def __init__(self, driver_init=None, driver=None):
        self.previous_total_reviews = None
        if driver is not None:
            self.driver = driver
            self.wait = WebDriverWait(driver, 10)
            self.short_wait = WebDriverWait(driver, 5, poll_frequency=0.5)
        elif driver_init is not None:
            self.driver = driver_init.driver
            self.wait = driver_init.wait
            self.short_wait = driver_init.short_wait
        else:
            raise ValueError("Either driver_init or driver must be provided.")

        self.previous_last_review_text = None
        self.reviews_df = pd.DataFrame(columns=['Review', 'Rating'])
        self.review_with_text_count = 0

    def return_to_first_review(self):
        try:
            first_review_element = self.wait.until(
                EC.visibility_of_element_located((By.XPATH, self.REVIEW_BLOCK_XPATH)))
            self.driver.execute_script("arguments[0].scrollIntoView(true);", first_review_element)
        except Exception as e:
            print(f"Error scrolling to first review: {e}")

    def load_reviews(self):
        self.previous_total_reviews = 0
        # Ensure visibility of initial reviews
        self.wait.until(EC.visibility_of_element_located((By.XPATH, self.REVIEW_BLOCK_XPATH)))
        while True:
            # Wait for any loading indicators to disappear, ensuring the page is ready
            self.short_wait.until(EC.invisibility_of_element((By.XPATH, self.LOADING_INDICATOR)))

            review_elements = self.driver.find_elements(By.XPATH, self.REVIEW_BLOCK_XPATH)
            current_total_reviews = len(review_elements)

            if not review_elements or current_total_reviews == self.previous_total_reviews:
                # If no new reviews are loaded or if the count hasn't changed, exit the loop
                break

            scroll_target_xpath = ".//div[4]"  # Adjust based on the precise element you need to scroll to
            scroll_target = review_elements[-1].find_element(By.XPATH, scroll_target_xpath)

            # Scroll to the specific target element to trigger loading more reviews
            self.driver.execute_script("arguments[0].scrollIntoView(true);", scroll_target)

            # Use WebDriverWait to explicitly wait for the number of reviews to increase
            try:
                WebDriverWait(self.driver, 10).until(
                    lambda driver: len(driver.find_elements(By.XPATH, self.REVIEW_BLOCK_XPATH)) > self.previous_total_reviews)
                # Update current count after waiting
                current_total_reviews = len(self.driver.find_elements(By.XPATH, self.REVIEW_BLOCK_XPATH))
            except TimeoutException:
                # If no new reviews are loaded within the timeout, assume all reviews are loaded
                break

            self.previous_total_reviews = current_total_reviews
            self.short_wait.until(EC.invisibility_of_element((By.XPATH, self.LOADING_INDICATOR)))
        print(f"Final count of reviews loaded: {current_total_reviews}")

    def click_translate_buttons(self):
        try:
            review_elements = self.driver.find_elements(By.XPATH, self.REVIEW_ELEMENTS_XPATH)
            for i, _ in enumerate(review_elements, start=1):
                translate_button_xpath = self.TRANSLATE_BUTTON_XPATH.format(i=i)
                translate_buttons = self.driver.find_elements(By.XPATH, translate_button_xpath)
                if translate_buttons:
                    translate_buttons[0].click()
        except Exception as e:
            print(f"Error clicking translate buttons: {e}")

    def click_more_buttons(self):
        try:
            more_buttons = self.driver.find_elements(By.XPATH, self.MORE_BUTTON_XPATH)
            for button in more_buttons:
                if button.is_displayed():
                    self.driver.execute_script("arguments[0].scrollIntoView();", button)
                    try:
                        button.click()
                    except Exception as e:
                        print(f"Skipping a non-interactable 'More' button: {e}")
        except Exception as e:
            print(f"Error clicking 'More' buttons: {e}")

    @staticmethod
    def extract_rating(aria_label):
        # Assuming the format is "Ocijenjeno s 5,0 od ukupno 5"
        # Adjust the regular expression to match this format
        match = re.search(r"Ocijenjeno s (\d+,\d+|\d+) od ukupno 5", aria_label)
        if match:
            # Replace comma with dot for float conversion if necessary
            rating_str = match.group(1).replace(',', '.')
            return float(rating_str)
        else:
            return None

    def collect_reviews(self):
        common_parents = self.driver.find_elements(By.XPATH, self.REVIEW_BLOCK_XPATH)
        print(f"Found {len(common_parents)} common parent elements.")

        # Initialize count of non-empty reviews collected
        count_non_empty_reviews = 0

        for i, common_parent_element in enumerate(common_parents, start=1):
            try:
                review_element = common_parent_element.find_element(By.XPATH, ".//div[contains(@class, 'OA1nbd')]")
                rating_element = common_parent_element.find_element(By.XPATH,
                                                                    ".//div[@role='img'][contains(@aria-label, 'Ocijenjeno s')]")

                review_text = review_element.text.strip()
                if review_text:  # Ensure review text is not empty
                    rating_text = rating_element.get_attribute('aria-label')
                    rating = self.extract_rating(rating_text)  # Extract numerical rating

                    if rating is not None:  # Check if rating extraction was successful
                        # Append review and rating to DataFrame
                        self.reviews_df.loc[len(self.reviews_df)] = [review_text, rating]
                        count_non_empty_reviews += 1  # Increment count of collected reviews

            except NoSuchElementException:
                print(f"Review or rating not found for block {i}. Skipping...")

        print(f"Collection completed. Total reviews collected: {count_non_empty_reviews}")

    def save_reviews_to_csv(self, filename='reviews_and_ratings.csv'):
        project_root = os.path.dirname(os.getcwd())
        directory = os.path.join(project_root, 'data', 'raw')

        if not os.path.exists(directory):
            os.makedirs(directory)

        filepath = os.path.join(directory, filename)

        try:
            self.reviews_df.to_csv(filepath, index=False)
            print(f"Saved reviews to {filepath}")
        except Exception as e:
            print(f"Error saving reviews to CSV: {e}")


In [76]:
review_scraper = ReviewScraper(driver=driver)

In [77]:
review_scraper.load_reviews()

Final count of reviews loaded: 2143


In [78]:
review_scraper.click_more_buttons()

In [79]:
review_scraper.collect_reviews()

Found 2223 common parent elements.
Review or rating not found for block 1734. Skipping...
Review or rating not found for block 1795. Skipping...
Review or rating not found for block 1803. Skipping...
Review or rating not found for block 1809. Skipping...
Review or rating not found for block 1816. Skipping...
Review or rating not found for block 1825. Skipping...
Review or rating not found for block 1839. Skipping...
Review or rating not found for block 1841. Skipping...
Review or rating not found for block 1845. Skipping...
Review or rating not found for block 1850. Skipping...
Review or rating not found for block 1853. Skipping...
Review or rating not found for block 1892. Skipping...
Review or rating not found for block 1904. Skipping...
Review or rating not found for block 1908. Skipping...
Review or rating not found for block 1917. Skipping...
Review or rating not found for block 1931. Skipping...
Review or rating not found for block 1935. Skipping...
Review or rating not found for

In [57]:
review_scraper.save_reviews_to_csv()

Saved reviews to C:\Users\ivan.zeljeznjak\DataspellProjects\RedditSentimentAnalysis\data\raw\reviews_and_ratings.csv


todo: dodaj neki counter za % reviewa koji su collectani, usporedi loadanje reviewe sa ukupnim 